In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# import chart_studio.plotly as py
import plotly.graph_objs as go
# from plotly_offline import plot

#for offline plotting
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)

In [ ]:
tesla=pd.read_csv('/kaggle/input/tesla-stock-price/Tesla.csv - Tesla.csv.csv')
tesla.head()

In [ ]:
tesla.info()

In [ ]:
tesla['Date']=pd.to_datetime(tesla['Date'])

In [ ]:
print(f'Dataframe contains stock prices between {tesla.Date.min()} {tesla.Date.max()}')
print(f'Total days = {(tesla.Date.max() - tesla.Date.min()).days} days')

In [ ]:
tesla.describe()

In [ ]:
tesla[['Open','High','Low','Close','Adj Close']].plot(kind='box')

In [ ]:
# Stteing the layout for our plot
layout = go.Layout(
title='Stock Prices of Tesla',xaxis=dict(title='Date',titlefont=dict(
family='Courier New,monospace',size=18,color='#7f7f7f')
                                        ),
    yaxis=dict(title='Price',
              titlefont=dict(family='Courier New,monospace',size=18,color='#7f7f7f')
              )
)

tesla_data=[{'x':tesla['Date'], 'y':tesla['Close']}]
plot=go.Figure(data=tesla_data,layout=layout)

In [ ]:
iplot(plot)

In [ ]:
# Building the regression model
from sklearn.model_selection import train_test_split

# For preprocessing 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# for model evaluation
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

In [ ]:
# split the data into train and test sets
x=np.array(tesla.index).reshape(-1,1)
y=tesla['Close']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=101)

In [ ]:
# Feature Scaling
scaler = StandardScaler().fit(x_train)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model=LinearRegression()
model.fit(x_train,y_train)

In [ ]:
# Plot actual and predicted values for train dataset
trace0=go.Scatter(
x=x_train.T[0],
y=y_train,
mode='markers',
name='Actual')

trace1=go.Scatter(
x=x_train.T[0],
y=model.predict(x_train).T,
mode='lines',
name='Predicted')

tesla_data=[trace0,trace1]
layout.xaxis.title.text='Day'
plot2=go.Figure(data=tesla_data,layout=layout)

In [ ]:
iplot(plot2)

In [ ]:
# Calculate scores for model evaluation
scores=f'''
{'Metric'.ljust(10)}{'Train'.center(20)}{'Test'.center(20)}
{'r2_score'.ljust(10)}{r2_score(y_train,model.predict(x_train))}\t{r2_score(y_test,model.predict(x_test))}
{'MSE'.ljust(10)}{mse(y_train,model.predict(x_train))}\t{mse(y_test,model.predict(x_test))}
'''
print(scores)

In [ ]:
data=pd.read_csv('/kaggle/input/tesla-stock-price/Tesla.csv - Tesla.csv.csv')

In [ ]:
data['Close']=pd.to_numeric(data.Close,errors='coerce')
data=data.dropna()
trainData=data.iloc[:,4:5].values

In [ ]:
sc=MinMaxScaler(feature_range=(0,1))
trainData= sc.fit_transform(trainData)
trainData.shape

In [ ]:
x_train=[]
y_train=[]
for i in range (60,1149):
    x_train.append(trainData[i-60:i,0])
    y_train.append(trainData[i,0])
x_train,y_train=np.array(x_train),np.array(y_train)    

In [ ]:
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))#adding the batch_size axis
x_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
model=Sequential()

model.add(LSTM(units=100,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))

model.add(LSTM(units=100,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=100,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=100,return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=1))
model.compile(optimizer='adam',loss="mean_squared_error")

In [ ]:
hist=model.fit(x_train,y_train,epochs=20,batch_size=32,verbose=2)

In [ ]:
plt.plot(hist.history['loss'])
plt.title('Training model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'],loc='upper left')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, explained_variance_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, SimpleRNN, GRU, BatchNormalization
import math
from sklearn.model_selection import train_test_split



dataset=pd.read_csv('/kaggle/input/tesla-stock-price/Tesla.csv - Tesla.csv.csv',index_col='Date', parse_dates=True)

In [ ]:
data = dataset[['Close']]
data.head()

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))
data = scaler.fit_transform(data)
data

In [ ]:
timesteps=60
features=1

In [ ]:
def split_data(data,timesteps,data_len):
    x,y=[],[]
    for i in range(timesteps,data_len):
        x.append(data[i-timesteps:i,:])
        y.append(data[i,:])
    return np.asarray(x),np.asarray(y)

In [ ]:
data_train,data_test=train_test_split(data,test_size=0.2,shuffle=False)
data_train,data_valid=train_test_split(data_train,test_size=0.2,shuffle=False)

print(data_train.shape,data_valid.shape,data_test.shape,sep='\n')

In [ ]:
x_train,y_train=split_data(data_train,timesteps,data_train.shape[0])

x_test,y_test = split_data(data_test,timesteps,data_test.shape[0])

x_valid,y_valid = split_data(data_valid,timesteps,data_valid.shape[0])

print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
print(x_valid.shape,x_valid.shape)

In [ ]:
model=Sequential()
model.add(SimpleRNN(60,activation='relu',return_sequences=True,input_shape=(timesteps,features)))
model.add(BatchNormalization())
model.add(SimpleRNN(60,activation='relu',return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(60))
model.add(BatchNormalization())
model.add(Dense(30))
model.add(Dense(1))

model.compile(optimizer='adam',loss='mse',metrics=['mae'])


In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=200, validation_data=(x_valid, y_valid))

In [ ]:
mse,mae=model.evaluate(x_test,y_test)

In [ ]:
y_pred=model.predict(x_test)
y_pred=scaler.inverse_transform(y_pred)
y_test=scaler.inverse_transform(y_test)

In [ ]:
rmse = np.sqrt(((y_test - y_pred)**2).mean())
rmse

In [ ]:
import math
def report(y_test,y_pred):
    print("RMSE: ", math.sqrt(mean_squared_error(y_test,y_pred)))
    print("MSE: ", mean_squared_error(y_test,y_pred))
    print("MAE: ", mean_absolute_error(y_test,y_pred))
    print("Explained variance regression score:", 
          explained_variance_score(y_test, y_pred))
    print('R2: ', r2_score(y_test, y_pred))

In [ ]:
report(y_test,y_pred)

In [ ]:
actual = dataset[['Close']]

pred   = dataset[['Close']].iloc[data.shape[0] - y_pred.shape[0]:, :]

pred[['Predict']] = y_pred
plt.figure(figsize=(16, 8))
plt.plot(actual, label='Actual')
plt.plot(pred[['Predict']], label='Predict')
plt.legend()
plt.show()